In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Datasets/fer2013.csv')

In [ ]:
print(data)

       emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
...        ...                                                ...          ...
35882        6  50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...  PrivateTest
35883        3  178 174 172 173 181 188 191 194 196 199 200 20...  PrivateTest
35884        0  17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...  PrivateTest
35885        3  30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...  PrivateTest
35886        2  19 13 14 12 13 16 21 33 50 57 71 84 97 108 122...  PrivateTest

[35887 rows x 3 columns]


In [ ]:
data.dtypes

emotion     int64
pixels     object
Usage      object
dtype: object

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []

for index, row in data.iterrows():
  k = row['pixels'].split(' ')
  if row['Usage'] == 'Training':
    X_train.append(np.array(k, dtype=float))
    y_train.append(row['emotion'])
  
  elif row['Usage'] == 'PublicTest':
    X_test.append(np.array(k, dtype=float))
    y_test.append(row['emotion'])
    

In [ ]:
type(X_test)

numpy.ndarray

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

y_train = np_utils.to_categorical(y_train, num_classes = 7)
y_test= np_utils.to_categorical(y_test, num_classes=7)


In [ ]:
plt.imshow(X_train[0] ,cmap=plt.cm.binary)
plt.xlabel(class_names[y_train[IMG_INDEX][0]])
plt.show()

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [ ]:
print(len(X_train[0][0]))

48


In [ ]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(48,48,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.summary()
#print(X_train)

In [ ]:
sgd = SGD(learning_rate=0.006)
model.compile(optimizer= sgd, loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=35, epochs=10, verbose=1, validation_data=(X_test, y_test))

In [ ]:
print(X_train.shape, X_train.dtype)
print(y_train.shape, y_train.dtype)
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,11)
plt.plot(epochs, loss_train, 'r', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,11)
plt.plot(epochs, loss_train, 'r', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test,y_test)
print(loss_and_metrics)

In [ ]:
model_json = model.to_json()
with open("/content/drive/My Drive/SavedModel/EmoDetect/modelnew.json", "w") as json_file:
  json_file.write(model_json)
model.save_weights("/content/drive/My Drive/SavedModel/EmoDetect/modelnew.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
!pip install git+git://github.com/PnS2019/pnslib.git

In [ ]:
import numpy as np
import os
from keras.models import model_from_json
import cv2
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from pnslib import utils

In [ ]:
model = model_from_json(open("/content/drive/My Drive/SavedModel/EmoDetect/model.json", "r").read())
model.load_weights('/content/drive/My Drive/SavedModel/EmoDetect/model.h5')

In [ ]:
face_haar_cascade = cv2.CascadeClassifier(utils.get_haarcascade_path('haarcascade_frontalface_default.xml'))
emotion_detection = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

In [ ]:
for file in os.listdir('/content/drive/My Drive/Datasets/EmoDetectTest'):
  fname = "/content/drive/My Drive/Datasets/EmoDetectTest/"+file
  img = cv2.imread(fname)
  
  cvt_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  face = face_haar_cascade.detectMultiScale(cvt_img, 1.3, 5)
  for (x,y,w,h) in face:
    cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0, 0), 2)
    roi_gray = cvt_img[y:y+h, x:x+w]
    roi_gray = cv2.resize(roi_gray,(48, 48))
  
    img_pix = img_to_array(roi_gray)
    img_pix = np.expand_dims(img_pix, axis = 0)
    #img_pix = img_pix / 255.0

    predict = model.predict(img_pix)

    emotion_prediction = emotion_detection[np.argmax(predict)]
    cv2.putText(img,emotion_prediction,(x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.imshow(img)
  plt.show()